# Attempt to do the experiments

Attempt to do the experiments with the original Glass dataset.

## Part I

#### Imports

In [42]:
import pandas as pd
import tensorflow as tf

from milp import codify_network
from teste import get_minimal_explanation

#### Opening the dataset

In [43]:
data = pd.read_csv('datasets/glass/glass.tsv', sep='\t')
data

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,target
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
200,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7
201,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7
202,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7
203,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7


In [44]:
data['target'].nunique()

5

#### Normalizing the data

<!-- The normalization will be simply to shrink all values to be from -1 to 1.
The minimum value will be -1; the maximum, 1.
To shrink I'll simply divide every value be the whole range of the feature. -->

I'll bring the mean to 0 by translating the data by the mean.

In [45]:
for column_name in data.columns[:-1]:
    data[column_name] -= data[column_name].mean()

In [46]:
data

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,target
0,0.002605,0.286537,1.744927,-0.348341,-0.846537,-0.458878,-0.189415,-0.182732,-0.059512,1
1,-0.000795,0.536537,0.854927,-0.088341,0.103463,-0.038878,-1.109415,-0.182732,-0.059512,1
2,-0.002225,0.176537,0.804927,0.091659,0.363463,-0.128878,-1.159415,-0.182732,-0.059512,1
3,-0.000745,-0.143463,0.944927,-0.158341,-0.016537,0.051122,-0.719415,-0.182732,-0.059512,1
4,-0.000985,-0.083463,0.874927,-0.208341,0.453463,0.031122,-0.869415,-0.182732,-0.059512,1
...,...,...,...,...,...,...,...,...,...,...
200,-0.002175,0.786537,-2.745073,1.431659,-0.016537,-0.438878,0.240585,0.877268,-0.059512,7
201,-0.001555,1.566537,-2.745073,0.541659,0.433463,-0.518878,-0.539415,1.407268,-0.059512,7
202,0.002245,1.006537,-2.745073,0.571659,0.793463,-0.518878,-0.499415,1.457268,-0.059512,7
203,-0.001895,1.026537,-2.745073,0.491659,0.983463,-0.518878,-0.459415,1.387268,-0.059512,7


Now I'll try to shrink the ranges

In [47]:
for column_name in data.columns[:-1]:
	length = data[column_name].max() - data[column_name].min()
	data[column_name] /= length

Now the data is between -1 and 1, and the mean is 0.

This normalization guarantees the algorithm will stretch each range more or less in the same time.


## Part II

#### Splitting the dataset

I'll split the dataset for training and testing purposes

In [48]:
from sklearn.model_selection import train_test_split

In [49]:
x = data.drop('target', axis=1)
y = data['target']

In [50]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

## Part III

#### Generating the network

In [59]:
num_classes = 5
num_neurons = 20
num_hidden_layers = 4

In [60]:
y_train_ohe = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_ohe = tf.keras.utils.to_categorical(y_test, num_classes)

IndexError: index 7 is out of bounds for axis 1 with size 5

In [55]:
from time import time 

keras_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=[x_train.shape[1]]),
])

for _ in range(num_hidden_layers):
    keras_model.add(tf.keras.layers.Dense(num_neurons, activation='relu'))

keras_model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

keras_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'], )

model_path = f'datasets/dir_path/model_{num_hidden_layers}layers_glass.h5'

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
ck = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True)

start = time()
keras_model.fit(x_train, y_train_ohe, batch_size=4, epochs=100, validation_data=(x_test, y_test_ohe), verbose=2, callbacks=[ck, es])
print(f'Tempo de Treinamento: {time()-start}')

keras_model = tf.keras.models.load_model(model_path)
print('Resultado Treinamento')
keras_model.evaluate(x_train, y_train_ohe, verbose=2)

print('Resultado Teste')
keras_model.evaluate(x_test, y_test_ohe, verbose=2)

Epoch 1/100
39/39 - 2s - loss: 2.0341 - accuracy: 0.3660 - val_loss: 1.9807 - val_accuracy: 0.3462 - 2s/epoch - 55ms/step
Epoch 2/100
39/39 - 0s - loss: 1.8872 - accuracy: 0.3791 - val_loss: 1.7835 - val_accuracy: 0.3462 - 145ms/epoch - 4ms/step
Epoch 3/100
39/39 - 0s - loss: 1.6092 - accuracy: 0.3791 - val_loss: 1.4707 - val_accuracy: 0.3462 - 126ms/epoch - 3ms/step
Epoch 4/100
39/39 - 0s - loss: 1.3521 - accuracy: 0.4641 - val_loss: 1.2577 - val_accuracy: 0.4038 - 201ms/epoch - 5ms/step
Epoch 5/100
39/39 - 0s - loss: 1.1725 - accuracy: 0.5098 - val_loss: 1.1193 - val_accuracy: 0.4038 - 142ms/epoch - 4ms/step
Epoch 6/100
39/39 - 0s - loss: 1.0569 - accuracy: 0.5882 - val_loss: 1.0574 - val_accuracy: 0.4231 - 196ms/epoch - 5ms/step
Epoch 7/100
39/39 - 0s - loss: 1.0073 - accuracy: 0.5425 - val_loss: 1.0184 - val_accuracy: 0.4423 - 242ms/epoch - 6ms/step
Epoch 8/100
39/39 - 0s - loss: 0.9717 - accuracy: 0.5621 - val_loss: 1.0080 - val_accuracy: 0.4808 - 188ms/epoch - 5ms/step
Epoch 9/10

[0.9792033433914185, 0.692307710647583]

In [56]:
(mp_model, output_bounds) = codify_network(keras_model, data, 'fischetti', relax_constraints=False)

#### Loading the model

In [58]:
# i = 134 is also a nice value to study
i = 138
print('i =', i)
network_input = data[i, :-1]
network_input = tf.reshape(tf.constant(network_input), [1, -1])
network_output = keras_model.predict(tf.constant(network_input))[0]
network_output = tf.argmax(network_output)

predictions = keras_model.predict(tf.constant(network_input))[0, 0]

print(f'Predictions: (ndarray[ndarray[{type(predictions)}]])', predictions)
classification = network_output.numpy()
print(f'Network output: ({type(classification)})', classification)

i = 138


InvalidIndexError: (138, slice(None, -1, None))